In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,turkcell=şahan gökbakaravea=ata demirervodafon...,0
1,dini imanı para olan turkcell fizy ile ilgili ...,0
2,"@zeyrekadin yok ""hat değiştirmiş"" adama sahtek...",1
3,avea gibi allah bin belanı versin yarin ilk iş...,1
4,oha lan turkcell 'den 14 gb internet geldi ay ...,1
...,...,...
9121,turkcell salla kazan kampanyası 2017,1
9122,çok başarılı turkcell müsteri hizmetleri'nin ç...,0
9123,ben platinum paketten ciktim siz tabii ki 6gb ...,0
9124,şuan o dört çeker diye diye övdüğünüz turkcell...,0


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
def tf_idf (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    tokenizer = Tokenizer(num_words=1000) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    vocabulary=[]
    for key in word_index.keys():
        vocabulary.append(key)

    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                     ('tfid', TfidfTransformer())]).fit(X_train)

    res = dict(zip(vocabulary, pipe['tfid'].idf_))

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    
    skipped_words = 0
    embedding_dim = 1 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = res[word] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [7]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= tf_idf (X_train,y_train,X_test,y_test)
    
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation='relu',return_sequences= True))
    model.add(SimpleRNN(256,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6998 - acc: 0.5209 - precision: 0.5214 - recall: 0.5468
Epoch 2/50
257/257 [==============================] - 8s 33ms/step - loss: 0.6928 - acc: 0.5180 - precision: 0.5186 - recall: 0.5449 3s - loss: 0.6932 - acc: 0.5155 - pr
Epoch 3/50
257/257 [==============================] - 8s 33ms/step - loss: 0.6908 - acc: 0.5265 - precision: 0.5266 - recall: 0.5558
Epoch 4/50
257/257 [==============================] - 8s 31ms/step - loss: 0.6889 - acc: 0.5360 - precision: 0.5378 - recall: 0.5337
Epoch 5/50
257/257 [==============================] - 8s 31ms/step - loss: 0.6873 - acc: 0.5384 - precision: 0.5420 - recall: 0.5153
Epoch 6/50
257/257 [==============================] - 8s 30ms/step - loss: 0.6868 - acc: 0.5399 - precision: 0.5417 - recall: 0.5371
Epoch 7/50
257/257 [==============================] - 11s 43ms/step 

Epoch 6/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6857 - acc: 0.5385 - precision_1: 0.5416 - recall_1: 0.4898
Epoch 7/50
257/257 [==============================] - 11s 44ms/step - loss: 0.6844 - acc: 0.5486 - precision_1: 0.5499 - recall_1: 0.5259
Epoch 8/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6834 - acc: 0.5473 - precision_1: 0.5513 - recall_1: 0.4983
Epoch 9/50
257/257 [==============================] - 11s 44ms/step - loss: 0.6820 - acc: 0.5591 - precision_1: 0.5586 - recall_1: 0.55440s - loss: 0.6819 - acc: 0.5582 - precision_1: 0.5582 - reca
Epoch 10/50
257/257 [==============================] - 11s 44ms/step - loss: 0.6817 - acc: 0.5586 - precision_1: 0.5636 - recall_1: 0.5115
Epoch 11/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6802 - acc: 0.5600 - precision_1: 0.5610 - recall_1: 0.5427
Epoch 12/50
257/257 [==============================] - 11s 44ms/step - loss: 0.6771 - acc: 0.5676 - precision_

257/257 [==============================] - 12s 47ms/step - loss: 0.6783 - acc: 0.5570 - precision_2: 0.5582 - recall_2: 0.5350
Epoch 14/50
257/257 [==============================] - 12s 46ms/step - loss: 0.6766 - acc: 0.5628 - precision_2: 0.5626 - recall_2: 0.5531
Epoch 15/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6759 - acc: 0.5622 - precision_2: 0.5638 - recall_2: 0.53820s - loss: 0.6758 - acc: 0.5615 - precision_2: 0.5
Epoch 16/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6730 - acc: 0.5712 - precision_2: 0.5703 - recall_2: 0.56753s - loss: 0.6707 - acc: 0.5751 - precisio - ETA: 2s - l
Epoch 17/50
257/257 [==============================] - 12s 46ms/step - loss: 0.6677 - acc: 0.5814 - precision_2: 0.5829 - recall_2: 0.56397s - loss: 0 - ETA: 1s - loss: 0.6674 - acc:
Epoch 18/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6663 - acc: 0.5814 - precision_2: 0.5834 - recall_2: 0.5612
Epoch 19/50
257/257 [=======

257/257 [==============================] - 11s 43ms/step - loss: 0.6713 - acc: 0.5758 - precision_3: 0.5828 - recall_3: 0.5288
Epoch 16/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6686 - acc: 0.5841 - precision_3: 0.5855 - recall_3: 0.5715
Epoch 17/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6636 - acc: 0.5877 - precision_3: 0.5957 - recall_3: 0.5422
Epoch 18/50
257/257 [==============================] - 11s 43ms/step - loss: 0.6613 - acc: 0.5871 - precision_3: 0.5902 - recall_3: 0.5659
Epoch 19/50
257/257 [==============================] - 10s 38ms/step - loss: 0.6559 - acc: 0.5948 - precision_3: 0.6012 - recall_3: 0.55950s - loss: 0.6565 - acc: 0.5935 - precision_3: 0.6002 - recall_3
Epoch 20/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6508 - acc: 0.6065 - precision_3: 0.6157 - recall_3: 0.5632
Epoch 21/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6473 - acc: 0.6049 - precision_3: 0.6

257/257 [==============================] - 12s 48ms/step - loss: 0.6700 - acc: 0.5859 - precision_4: 0.5887 - recall_4: 0.59230s - loss: 0.6701 - acc: 0.5859 - precision_4: 0.587
Epoch 16/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6650 - acc: 0.5858 - precision_4: 0.5894 - recall_4: 0.58741s - loss: 0.6659 - acc: 0.5842 - precision_
Epoch 17/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6628 - acc: 0.5913 - precision_4: 0.5895 - recall_4: 0.6229
Epoch 18/50
257/257 [==============================] - 12s 48ms/step - loss: 0.6586 - acc: 0.5926 - precision_4: 0.5958 - recall_4: 0.59610s - loss: 0.6586 - acc: 0.5924 - precision_4: 0.5957 - recall_4: 0.
Epoch 19/50
257/257 [==============================] - 12s 47ms/step - loss: 0.6551 - acc: 0.6005 - precision_4: 0.6031 - recall_4: 0.6070
Epoch 20/50
257/257 [==============================] - 12s 45ms/step - loss: 0.6472 - acc: 0.6195 - precision_4: 0.6216 - recall_4: 0.6268
Epoch 21/50
25

257/257 [==============================] - 7s 28ms/step - loss: 0.6561 - acc: 0.5933 - precision_5: 0.6012 - recall_5: 0.5463
Epoch 19/50
257/257 [==============================] - 7s 27ms/step - loss: 0.6504 - acc: 0.6019 - precision_5: 0.6076 - recall_5: 0.5676
Epoch 20/50
257/257 [==============================] - 7s 28ms/step - loss: 0.6450 - acc: 0.6093 - precision_5: 0.6155 - recall_5: 0.5754 1s - loss: 0.6459 - acc: 0
Epoch 21/50
257/257 [==============================] - 7s 28ms/step - loss: 0.6428 - acc: 0.6120 - precision_5: 0.6178 - recall_5: 0.5803
Epoch 22/50
257/257 [==============================] - 7s 28ms/step - loss: 0.6327 - acc: 0.6246 - precision_5: 0.6305 - recall_5: 0.5961
Epoch 23/50
257/257 [==============================] - 7s 28ms/step - loss: 0.6235 - acc: 0.6292 - precision_5: 0.6340 - recall_5: 0.6057
Epoch 24/50
257/257 [==============================] - 7s 28ms/step - loss: 0.6196 - acc: 0.6395 - precision_5: 0.6489 - recall_5: 0.6025 0s - loss: 0.6201 -

257/257 [==============================] - 8s 32ms/step - loss: 0.6398 - acc: 0.6158 - precision_6: 0.6196 - recall_6: 0.6006
Epoch 24/50
257/257 [==============================] - 6s 24ms/step - loss: 0.6347 - acc: 0.6202 - precision_6: 0.6278 - recall_6: 0.5911
Epoch 25/50
257/257 [==============================] - 6s 22ms/step - loss: 0.6226 - acc: 0.6322 - precision_6: 0.6438 - recall_6: 0.5926
Epoch 26/50
257/257 [==============================] - 8s 33ms/step - loss: 0.6156 - acc: 0.6384 - precision_6: 0.6477 - recall_6: 0.6077
Epoch 27/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6080 - acc: 0.6446 - precision_6: 0.6569 - recall_6: 0.6062
Epoch 28/50
257/257 [==============================] - 8s 33ms/step - loss: 0.6021 - acc: 0.6553 - precision_6: 0.6610 - recall_6: 0.6384
Epoch 29/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5873 - acc: 0.6656 - precision_6: 0.6713 - recall_6: 0.6495
Epoch 30/50
257/257 [=========================

257/257 [==============================] - 8s 30ms/step - loss: 0.6235 - acc: 0.6344 - precision_7: 0.6336 - recall_7: 0.6335
Epoch 28/50
257/257 [==============================] - 8s 29ms/step - loss: 0.6166 - acc: 0.6346 - precision_7: 0.6348 - recall_7: 0.6303 3s - loss: 0.6105 - acc:
Epoch 29/50
257/257 [==============================] - 8s 29ms/step - loss: 0.6124 - acc: 0.6446 - precision_7: 0.6442 - recall_7: 0.6428
Epoch 30/50
257/257 [==============================] - 7s 29ms/step - loss: 0.6012 - acc: 0.6525 - precision_7: 0.6516 - recall_7: 0.6525
Epoch 31/50
257/257 [==============================] - 7s 29ms/step - loss: 0.5921 - acc: 0.6648 - precision_7: 0.6654 - recall_7: 0.6603
Epoch 32/50
257/257 [==============================] - 8s 30ms/step - loss: 0.5791 - acc: 0.6680 - precision_7: 0.6718 - recall_7: 0.6542 2s - loss: 0.5789 - acc: 0.6721 - precision_ - ETA: 1s - loss: 0.5797 - acc: 0.6
Epoch 33/50
257/257 [==============================] - 7s 29ms/step - loss: 0.

257/257 [==============================] - 8s 32ms/step - loss: 0.5595 - acc: 0.6855 - precision_8: 0.6860 - recall_8: 0.6838
Epoch 32/50
257/257 [==============================] - 6s 22ms/step - loss: 0.5511 - acc: 0.6948 - precision_8: 0.6948 - recall_8: 0.6943
Epoch 33/50
257/257 [==============================] - 5s 21ms/step - loss: 0.5351 - acc: 0.6995 - precision_8: 0.7027 - recall_8: 0.6911
Epoch 34/50
257/257 [==============================] - 6s 22ms/step - loss: 0.5219 - acc: 0.7064 - precision_8: 0.7070 - recall_8: 0.7043
Epoch 35/50
257/257 [==============================] - 5s 21ms/step - loss: 0.5065 - acc: 0.7215 - precision_8: 0.7175 - recall_8: 0.7301 
Epoch 36/50
257/257 [==============================] - 5s 21ms/step - loss: 0.5077 - acc: 0.7241 - precision_8: 0.7205 - recall_8: 0.7320 0s - loss: 0.5047 - acc: 0.7252 - precision_8:
Epoch 37/50
257/257 [==============================] - 6s 21ms/step - loss: 0.5051 - acc: 0.7224 - precision_8: 0.7216 - recall_8: 0.723

257/257 [==============================] - 5s 20ms/step - loss: 0.5524 - acc: 0.6914 - precision_9: 0.7013 - recall_9: 0.6673
Epoch 34/50
257/257 [==============================] - 5s 20ms/step - loss: 0.5439 - acc: 0.6984 - precision_9: 0.7078 - recall_9: 0.6763 2s
Epoch 35/50
257/257 [==============================] - 5s 20ms/step - loss: 0.5353 - acc: 0.7051 - precision_9: 0.7128 - recall_9: 0.6875 3s - loss: 0.5
Epoch 36/50
257/257 [==============================] - 5s 20ms/step - loss: 0.5184 - acc: 0.7201 - precision_9: 0.7256 - recall_9: 0.7084
Epoch 37/50
257/257 [==============================] - 5s 20ms/step - loss: 0.5073 - acc: 0.7263 - precision_9: 0.7311 - recall_9: 0.7165
Epoch 38/50
257/257 [==============================] - 5s 20ms/step - loss: 0.4969 - acc: 0.7333 - precision_9: 0.7423 - recall_9: 0.7150
Epoch 39/50
257/257 [==============================] - 5s 20ms/step - loss: 0.4787 - acc: 0.7471 - precision_9: 0.7540 - recall_9: 0.7340
Epoch 40/50
257/257 [=======

In [8]:
def Average(lst):
    return sum(lst) / len(lst)

In [9]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.536271
test precision: 0.537052
test recall: 0.536509
test f1_score: 0.536099
